In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

## 1.Load data

This data set have 42000 train data, and 28000 test data.

In [7]:
def load_data():
    train = pd.read_csv('../input/train.csv')
    test = pd.read_csv('../input/test.csv')
    print("train shape:{}\ntest shape:{}".format(train.shape, test.shape))
    
    train_df = train.drop(['label'], axis=1)
    label = pd.get_dummies(train['label'])
    train_df = train_df.applymap(lambda x: x/255)
    test = test.applymap(lambda x: x/255)
    
    x_train, x_valid, y_train, y_valid = train_test_split(train_df, label, test_size=0.2, random_state=2017)
    print("x_train shape:"+str(x_train.shape))
    print("x_valid shape:"+str(x_valid.shape))
    
    return x_train, y_train, x_valid, y_valid, test

In [8]:
x_train, y_train, x_valid, y_valid, x_test = load_data()

train shape:(42000, 785)
test shape:(28000, 784)
x_train shape:(33600, 784)
x_valid shape:(8400, 784)


## 2.Tensorflow classifer

### 2.1 Tensorflow high-level API

Tensorflow have high-level API:tf.contrib.learn, this API can use with sklearn.

**DNNClassifier**
    - hidden_unit: NN cunstruct
    - n_classes: output class number
    - feature_columns: train feature

In [40]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(x_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,300,100],
                                        n_classes=10,
                                        feature_columns=feature_columns)

dnn_clf.fit(x=x_train, y=y_train, batch_size=50, steps=40000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_session_config': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_is_chief': True, '_task_id': 0, '_num_worker_replicas': 0, '_task_type': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f24f9c79898>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_environment': 'local', '_model_dir': '/tmp/tmpy942inc1', '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpy942inc1/model.ckpt.
INFO:tensorflow:loss = 2.3489, step = 1
INFO:tensorflow:global_step/sec: 240.232
INFO:tensorflow:loss = 0.49387, step = 101 (0.416 sec)
INFO:tensorflow:global_step/sec: 267.539
INFO:tensorflow:loss = 0.51

DNNClassifier(params={'gradient_clip_norm': None, 'dropout': None, 'hidden_units': [300, 300, 100], 'embedding_lr_multipliers': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x7f24f9c797f0>, 'input_layer_min_slice_size': None, 'optimizer': None, 'activation_fn': <function relu at 0x7f254e050400>, 'feature_columns': (_RealValuedColumn(column_name='', dimension=784, default_value=None, dtype=tf.float64, normalizer=None),)})

In [41]:
from sklearn.metrics import accuracy_score

y_pre = list(dnn_clf.predict(x_valid))
print("accuracy score:" + str(accuracy_score(y_valid, y_pre)))

INFO:tensorflow:Restoring parameters from /tmp/tmpy942inc1/model.ckpt-40000
accuracy score:0.979761904762


In [42]:
pre = list(dnn_clf.predict(test))

INFO:tensorflow:Restoring parameters from /tmp/tmpy942inc1/model.ckpt-40000


## Tensorflow implement logistics regression

In [15]:
sess =tf.InteractiveSession()

x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

w = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, w) + b)

cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
tf.global_variables_initializer().run()

In [16]:
train_size = x_train.shape[0]

for i in range(1000):
    start = i*64 % train_size
    end = (i+1)*64 % train_size    
    #print(start, end)
    if start > end:
        start = 0
    
    batch_x = x_train[start:end]
    batch_y = y_train[start:end]  
    
    if i%100 == 0:
        print(cross_entropy.eval(feed_dict={x:batch_x, y_:batch_y}))
    sess.run(train_step,feed_dict={x:batch_x,y_:batch_y})

147.365
23.7593
9.87678
25.1089
23.5327
18.8673
21.9565
38.6417
18.5168
24.1314


In [21]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x: x_valid, y_: y_valid}))

0.910357


## Prediction

In [26]:
#y_pre = tf.nn.softmax(tf.matmul(x_test, w) + b)
y_pre = sess.run(tf.nn.softmax(tf.matmul(x, w) + b), feed_dict={x:x_test})
pre_label = np.argmax(y_pre, axis=1)
print(pre_label)

[2 0 9 ..., 3 9 2]


# Submit

In [43]:
from datetime import datetime

submit_df= pd.read_csv('../input/sample_submission.csv')
submit_df.Label = pre
submit_df.to_csv('../sub/tensorflow_dnn.csv',index=None,encoding='utf-8')

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
